In [1]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostClassifier
%matplotlib inline




le = preprocessing.LabelEncoder()

### un modelo precario, solo usamos el largo de mi texto.

### Cargo mi texto

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


### arreglo los que estan mal 

In [3]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train.loc[train['id'].isin(ids_with_target_error),'target'] = 0

### agrego una columna con el largo del tweet

In [4]:
train['largo'] = train['text'].str.len()

In [5]:
train.sample(3)

,id,keyword,location,text,target,largo
410,592,arson,NaN,Arson suspect linked to 30 fires caught in Nor...,1,105
4242,6027,hazardous,"Portland, OR",HAZARD - HAZARDOUS CONDITION at SB I5 FWY AT /...,1,125
5558,7933,rainstorm,NaN,Robot_Rainstorm: We have two vacancies on the ...,0,100


In [6]:
train = train [ ['id','largo', 'target' ] ]

In [7]:
train.sample(3)

,id,largo,target
7279,10419,110,0
7170,10274,32,1
6922,9930,122,0


In [8]:
largos = train['largo'].values.reshape(-1,1)

In [9]:
targets = train['target']

In [10]:
largos

array([[ 69],
       [ 38],
       [133],
       ...,
       [ 65],
       [137],
       [ 94]])

In [11]:
modelo = KNeighborsClassifier(n_neighbors=7)

In [12]:
modelo.fit(largos,targets)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [13]:
preds = modelo.predict(largos)

### pruebo como va, hago mis pred sobre el test.

In [14]:
print('Accuracy Score : ' + str(accuracy_score(targets,preds)))
print('Precision Score : ' + str(precision_score(targets,preds)))
print('Recall Score : ' + str(recall_score(targets,preds)))
print('F1 Score : ' + str(f1_score(targets,preds)))

Accuracy Score : 0.577958754761592
Precision Score : 0.5085020242914979
Recall Score : 0.3858678955453149
F1 Score : 0.43877729257641923


#### Claramente este modelo es mediocre, como ejercicio busco para que neighbor entre 1 y 1000 tiene la mejor accuracy.

In [15]:
def accuracy(vecinos, targets, largos):
    
  modelo = KNeighborsClassifier(n_neighbors = vecinos)
  modelo.fit(largos,targets)
  preds = modelo.predict(largos)
  return accuracy_score(targets,preds)

### busco el maximo entre 0 y 1000

In [16]:
accuracy_maxima = 0
indice_maximo = 0
for x in range (1, 1001):
    accuracy_actual = accuracy(x,targets,largos)
    if (accuracy_actual > accuracy_maxima):
        accuracy_maxima = accuracy_actual
        indice_maximo = x

In [17]:
print(indice_maximo, accuracy_maxima )

93 0.6174963877577827


#### el numero de vecinos mas optimo es 93 y tiene una accuracy de 0.62. -> sobre el mismo set donde fue entrenado